# Preparations

In [ ]:
# Clone realESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Clone BSRGAN
!git clone https://github.com/cszn/BSRGAN.git

!rm -r SwinIR
# Clone SwinIR
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm

# Download the pre-trained models
!wget https://github.com/cszn/KAIR/releases/download/v1.0/BSRGAN.pth -P BSRGAN/model_zoo
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 755, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 755 (delta 5), reused 12 (delta 3), pack-reused 735
Receiving objects: 100% (755/755), 5.37 MiB | 19.18 MiB/s, done.
Resolving deltas: 100% (406/406), done.
/content/Real-ESRGAN/Real-ESRGAN
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
running develop
running egg_info
creating realesrgan.egg-info
writing realesrgan.egg-info/PKG-INFO
writing dependency_links to realesrgan.egg-info/dependency_links.txt
writing requirements to realesrgan.egg-info/requires.txt
writing to

# Data

In [ ]:
import os
import glob
from google.colab import files
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_test = zipfile.ZipFile('/content/drive/MyDrive/KHUGGLE/khuggle_test.zip')
zip_test.extractall('/content/khuggle_test')
 
zip_test.close()

In [ ]:
!rm -r BSRGAN/testsets/RealSRSet

shutil.copytree('/content/khuggle_test/', 'BSRGAN/testsets/RealSRSet/')

os.mkdir('results')

test_patch_wise = False

# Inference

In [ ]:
# BSRGAN
!rm -r results
if not test_patch_wise:
  %cd BSRGAN
  !python main_test_bsrgan.py
  %cd ..
  shutil.move('BSRGAN/testsets/RealSRSet_results_x4', 'results/BSRGAN')

# realESRGAN
if test_patch_wise:
  !python inference_realesrgan.py -n RealESRGAN_x4plus --input BSRGAN/testsets/RealSRSet -s 4 --output results/realESRGAN --tile 800
else:
  !python inference_realesrgan.py -n RealESRGAN_x4plus --input BSRGAN/testsets/RealSRSet -s 4 --output results/realESRGAN

# SwinIR-Large
if test_patch_wise:
  !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 4 --large_model --tile 640
else:
  !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 4 --large_model
shutil.move('results/swinir_real_sr_x4_large', 'results/SwinIR_large')
for path in sorted(glob.glob(os.path.join('results/SwinIR_large', '*.png'))):
  os.rename(path, path.replace('SwinIR.png', 'SwinIR_large.png')) # here is a bug in Colab file downloading: no same-name files

/content/Real-ESRGAN/Real-ESRGAN/BSRGAN
LogHandlers setup!
22-12-08 09:45:32.476 :       Model Name : BSRGAN
22-12-08 09:45:32.491 :           GPU ID : 0
[3, 3, 64, 23, 32, 4]
22-12-08 09:45:37.866 :       Input Path : testsets/RealSRSet
22-12-08 09:45:37.866 :      Output Path : testsets/RealSRSet_results_x4
22-12-08 09:45:37.867 : ---1 --> BSRGAN --> x4--> 0001.png
22-12-08 09:45:45.898 : ---2 --> BSRGAN --> x4--> 0002.png
22-12-08 09:45:46.311 : ---3 --> BSRGAN --> x4--> 0003.png
22-12-08 09:45:46.727 : ---4 --> BSRGAN --> x4--> 0004.png
22-12-08 09:45:47.137 : ---5 --> BSRGAN --> x4--> 0005.png
22-12-08 09:45:47.548 : ---6 --> BSRGAN --> x4--> 0006.png
22-12-08 09:45:47.920 : ---7 --> BSRGAN --> x4--> 0007.png
22-12-08 09:45:48.340 : ---8 --> BSRGAN --> x4--> 0008.png
22-12-08 09:45:48.960 : ---9 --> BSRGAN --> x4--> 0009.png
22-12-08 09:45:49.374 : --10 --> BSRGAN --> x4--> 0010.png
22-12-08 09:45:49.795 : --11 --> BSRGAN --> x4--> 0011.png
22-12-08 09:45:50.211 : --12 --> BSRGAN 